### 先頭の2単語だけ固定し、2番目の単語を総当りして探索

In [1]:
ex_num = "005"

In [2]:
import os
# 環境によって変更
# os.chdir('/home/jovyan/work/notebook')

path = "../../output"
# path = "../model"

In [3]:
import numpy as np
import pandas as pd
import os
import time
import csv

from tqdm.notebook import tqdm

import random
from typing import List, Callable
import itertools, math
from typing import List, Tuple

from utils import PerplexityCalculator

### データ読み込み

In [4]:
df = pd.read_csv("../input/sample_submission.csv")

In [5]:
sub_df = df.copy()
sub_df["best_value"] = 0

In [6]:
df

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


### 関数

In [7]:
def simulated_annealing(
    scorer,
    pattern_df,
    max_iterations: int = 1000,
    BATCH_SIZE: int = 32,
    initial_temperature: float = 100.0,
    cooling_rate: float = 0.99
) -> Tuple[List[str], float]:
    """
    焼きなまし法で最適な順序を探索する。

    Args:
        scorer: 評価関数を提供するオブジェクト。
        pattern_df: パターンのデータフレーム。
        max_iterations (int): 最大反復回数。
        BATCH_SIZE (int): バッチサイズ。
        initial_temperature (float): 初期温度。
        cooling_rate (float): 冷却率。

    Returns:
        solution_df: 最適な順序のデータフレーム。
    """
    
    solution_df = pattern_df.copy()
    solution_df["score"] = solution_df["score"].astype(float)
    
    words_len = len(solution_df.iloc[0]["text"].split())
    pattern_len = len(solution_df)
    
    temperature = initial_temperature
    
    perms = []
    best_score = 10000
    global_best_sequence = ""
    
    for iteration in tqdm(range(max_iterations)):
        # 1行ずつ処理する
        for idx in range(pattern_len):
            current_sequence = solution_df.iloc[idx]["text"].split()
            
            neighbor_sequence = current_sequence[:]
            
            # 最初と最後の単語を固定し、ランダムに交換する
            i, j = random.sample(range(2, words_len), 2)
            neighbor_sequence[i], neighbor_sequence[j] = neighbor_sequence[j], neighbor_sequence[i]
            
            perms.append(" ".join(neighbor_sequence))
        
        # 評価値をまとめて計算
        neighbor_values = scorer.get_perplexity(perms, batch_size=min(pattern_len, BATCH_SIZE))

        # print(f"neighbor_values = {neighbor_values}")
        # 焼きなましによる評価値更新
        for idx in range(pattern_len):
            perm = perms[idx]
            neighbor_value = neighbor_values[idx]
            current_value = solution_df.iloc[idx]["score"]
            # print(f"current_value = {current_value}, neighbor_value = {neighbor_value}")
            
            # 確率的に次の状態を受け入れるかを決定
            delta = neighbor_value - current_value
            acceptance_probability = math.exp(-delta / temperature) if delta > 0 else 1.0

            # ベスト解を更新
            if (random.random() < acceptance_probability) or (neighbor_value < current_value):
                current_sequence = perm.split(" ")
                current_value = neighbor_value
                
                solution_df.loc[solution_df.index[idx], "text"] = perm
                solution_df.loc[solution_df.index[idx], "score"] = current_value
                start_word = solution_df.iloc[idx]["start"]
                # print(f"start_word = {start_word}, end_word = {end_word}, New best = {current_value} with '{current_sequence}'")

            if current_value < best_score:
                best_score = current_value
                global_best_sequence = perm
                print(f"New best = {best_score} with '{perm}'")
                
        # バッチリセット
        perms = []

        # 温度を更新
        if iteration % 100 == 0:
            temperature *= cooling_rate
                    
        # 温度が非常に低くなったら終了
        if temperature < 1e-5:
            break

        # iteration100回ごとに現在の温度を表示
        if iteration % 100 == 0:
            print(f"Iteration {iteration}, Temperature = {temperature}")
            
    return solution_df, best_score, global_best_sequence
     

In [8]:
def search_optimal_permutation(id: int, scorer, max_iterations: int = 1000, BATCH_SIZE=64):
    
    df = pd.read_csv("../input/sub20241219.csv")
    
    # 文字列を単語に分割
    words = df.loc[id,"text"].split()
    
    # データをタプルのリストとして準備
    pattern_data = []
    
    words_len = len(words)
    for i in range(2, words_len):
        new_words = words[:]
        if i != 2:
            new_words[i], new_words[2] = new_words[2], new_words[i]
        
        # new_words[1]意外の単語をシャッフル
        random.shuffle(new_words[2:])
        
        pattern_data.append((new_words[0], " ".join(new_words), 10000))
            
    pattern_df = pd.DataFrame(pattern_data, columns=["start", "text", "score"])        

    start = time.time()
    
    # SAで最適な順序を探索
    solutions_df, best_value, best_sequence = simulated_annealing(
        scorer,
        pattern_df,
        max_iterations=max_iterations,
        BATCH_SIZE=BATCH_SIZE,
        initial_temperature=50.0,
        cooling_rate=0.997
    )
    
    # # optimized_dfのscore列の値が最小の行を取得
    # best_sequence = solutions_df.loc[solutions_df["score"].idxmin(), "text"]
    # best_value = solutions_df["score"].min()
    
    print(f"{id}th sample: {best_sequence}")
    print(f"Elapsed time: {time.time() - start:.2f} sec")
    print(f"Best value: {best_value}")
    
    return best_sequence, best_value, solutions_df


In [9]:
# 文字列型にのみダブルクォーテーションを付ける関数
def add_quotes_to_strings(value):
    if isinstance(value, str):  # 文字列型の場合
        return f'"{value}"'
    return value  # それ以外はそのまま

In [10]:
# LOAD GEMMA SCORER
scorer = PerplexityCalculator(f'{path}/gemma_2_9b')

cuda


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
# scorer.clear_gpu_memory()

In [12]:
for i in [1, 2]:
    
    print(f"Processing {i}th sample")
    best_sequence, best_value, solutions_df = search_optimal_permutation(
        id=i,
        scorer=scorer,
        max_iterations=2**18,
        BATCH_SIZE=256
    )
    
    print(best_sequence)
    sub_df.loc[i, "best_value"] = best_value
    sub_df.loc[i, "text"] = best_sequence
    
    display(solutions_df)
    solutions_df.to_csv(f"{path}/out/solutions_{i}_{ex_num}.csv", index=False, header=True)



Processing 1th sample


  0%|          | 0/262144 [00:00<?, ?it/s]

New best = 736.2446732757018 with 'reindeer sleep walk the night and bake mistletoe scrooge laugh chimney jump elf drive gingerbread family give advent fireplace ornament'
Iteration 0, Temperature = 49.85
New best = 705.279198037542 with 'reindeer sleep walk the night give and mistletoe scrooge laugh chimney jump elf bake gingerbread family drive advent fireplace ornament'
New best = 642.1640174789829 with 'reindeer sleep drive the night scrooge laugh mistletoe elf gingerbread walk and jump bake ornament family give advent fireplace chimney'
New best = 584.6969916200433 with 'reindeer sleep walk the night and drive mistletoe scrooge laugh chimney elf bake gingerbread family advent give fireplace jump ornament'
New best = 557.921766975976 with 'reindeer sleep walk the night and drive mistletoe scrooge laugh chimney elf bake gingerbread family ornament give fireplace jump advent'
Iteration 100, Temperature = 49.700450000000004
New best = 555.7466361419574 with 'reindeer sleep walk the ni

/tmp/ipykernel_118/3950568173.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '422.78974553481265' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  sub_df.loc[i, "best_value"] = best_value


,start,text,score
0,reindeer,reindeer sleep walk jump drive the mistletoe l...,564.498372
1,reindeer,reindeer sleep and drive mistletoe jump scroog...,498.168065
2,reindeer,reindeer sleep walk the night drive give and b...,509.981780
3,reindeer,reindeer sleep elf walk the night and drive sc...,462.533390
4,reindeer,reindeer sleep drive the elf and scrooge laugh...,547.130583
5,reindeer,reindeer sleep drive the night scrooge walk gi...,568.925788
6,reindeer,reindeer sleep walk the night and drive mistle...,422.789746
7,reindeer,reindeer sleep walk gingerbread bake elf laugh...,542.872779
8,reindeer,reindeer sleep drive the night give and bake l...,528.229712
9,reindeer,reindeer sleep walk drive the night give and b...,490.444685


Processing 2th sample


  0%|          | 0/262144 [00:00<?, ?it/s]

New best = 333.1503034969411 with 'jingle yuletide carol cheer grinch holiday holly naughty nice nutcracker ornament decorations gifts stocking magi sleigh polar beard chimney workshop'
Iteration 0, Temperature = 49.85
Iteration 100, Temperature = 49.700450000000004
Iteration 200, Temperature = 49.55134865
Iteration 300, Temperature = 49.40269460405
Iteration 400, Temperature = 49.25448652023785
Iteration 500, Temperature = 49.10672306067713
Iteration 600, Temperature = 48.9594028914951
Iteration 700, Temperature = 48.812524682820616
Iteration 800, Temperature = 48.66608710877215
Iteration 900, Temperature = 48.520088847445834
Iteration 1000, Temperature = 48.374528580903494
Iteration 1100, Temperature = 48.229404995160785
Iteration 1200, Temperature = 48.0847167801753
Iteration 1300, Temperature = 47.94046262983477
Iteration 1400, Temperature = 47.79664124194527
Iteration 1500, Temperature = 47.65325131821943
Iteration 1600, Temperature = 47.510291564264776
Iteration 1700, Temperature

,start,text,score
0,jingle,jingle yuletide carol holiday cheer holly naug...,329.268985
1,jingle,jingle yuletide carol holiday cheer holly stoc...,335.763234
2,jingle,jingle yuletide carol holiday cheer holly naug...,325.432885
3,jingle,jingle yuletide carol cheer sleigh workshop ch...,363.046618
4,jingle,jingle yuletide carol cheer holly naughty nice...,326.706593
5,jingle,jingle yuletide carol holiday cheer holly stoc...,334.454217
6,jingle,jingle yuletide carol cheer grinch holiday hol...,331.851474
7,jingle,jingle yuletide carol holiday cheer gifts orna...,354.636652
8,jingle,jingle yuletide carol holiday cheer holly stoc...,346.421503
9,jingle,jingle yuletide carol grinch naughty nice pola...,373.110641


In [13]:
sub_df

,id,text,best_value
0,0,advent chimney elf family fireplace gingerbrea...,0.000000
1,1,reindeer sleep walk the night and drive mistle...,422.789746
2,2,jingle yuletide carol cheer holly stocking chi...,315.420368
3,3,yuletide decorations gifts cheer holiday carol...,0.000000
4,4,hohoho candle poinsettia snowglobe peppermint ...,0.000000
5,5,advent chimney elf family fireplace gingerbrea...,0.000000


In [14]:
# 各セルに関数を適用
sub_df["text"] = sub_df["text"].astype(str)
sub_df = sub_df.applymap(add_quotes_to_strings)

/tmp/ipykernel_118/428122046.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  sub_df = sub_df.applymap(add_quotes_to_strings)


In [15]:
sub_df[["id", "text"]].to_csv(f"{path}/out/submission_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)
sub_df.to_csv(f"{path}/out/score_{ex_num}.csv", index=False, header=True, quoting=csv.QUOTE_NONE)

In [16]:
np.mean(sub_df["best_value"])

np.float64(123.0350188826679)